In [101]:
import requests
import pandas as pd
from tqdm import tqdm
from langchain_community.chat_models import ChatCohere
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# Loading Dataset

In [89]:
df=pd.read_csv("sentinelle_liste_sp.csv")
df = df.drop(0)
df = df[df["Regne"]=="Flore"].reset_index(drop=True)

In [90]:
df

,Regne,Categorie,Code_espece,Nom_francais,Nom_latin,Nom_anglais
0,Flore,Plantes aquatiques flottantes,CHATA,Châtaigne d'eau,Trapa natans,Water chestnut
1,Flore,Plantes aquatiques flottantes,FAUXN,Faux-nymphéa pelté,Nymphoides peltata,Yellow floatingheart
2,Flore,Plantes aquatiques flottantes,HYDRO,Hydrocharide grenouillette,Hydrocharis morsus-ranae,European frog-bit
3,Flore,Plantes aquatiques flottantes,JACYN,Jacinthe d'eau,Eichhornia crassipes,Water hyacinth
4,Flore,Plantes aquatiques flottantes,LAITU,Laitue d'eau,Pistia stratiotes,Water lettuce
5,Flore,Plantes aquatiques flottantes,SALVI,Salvinia,Salvinia spp.,Water moss
6,Flore,Plantes aquatiques submergées,CABOM,Cabomba de Caroline,Cabomba caroliniana,Fanwort
7,Flore,Plantes aquatiques submergées,ELODE,Élodée dense,Egeria densa,Brazilian waterweed
8,Flore,Plantes aquatiques submergées,HYDRI,Hydrille verticillé,Hydrilla verticillata,Hydrilla
9,Flore,Plantes aquatiques submergées,MYREP,Myriophylle à épis,Myriophyllum spicatum,Eurasian water-milfoil


# Sentinelle API 

In [91]:
def get_plant_info(code):
    url = "https://www.pub.enviroweb.gouv.qc.ca/SCC/Catalogue/ConsulterCatalogue.aspx/ObtenirFiche"
    headers = {
        # "Accept": "application/json, text/javascript, */*; q=0.01",
        # "Accept-Encoding": "gzip, deflate, br, zstd",
        # "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        # "Connection": "keep-alive",
        # "Content-Length": "16",
        # "Content-Type": "application/json; charset=UTF-8",
        # "Cookie": "_ga_W9S3681TN0=GS1.1.1678636939.2.1.1678636997.0.0.0; _ga_GTETXTSJ02=GS1.1.1704698258.1.0.1704698262.0.0.0; _clck=q56etf%7C2%7Cfk1%7C0%7C1533; _ga=GA1.1.233539128.1677610405; ASP.NET_SessionId=0exnh3gpwmgnvzzdlaekdtbp; .SCCEEEWEBAUTH=C35EDE4FC6A06FFA820194E624AD98A4C50772973A6BF0234DFBD52609AB24B8D3D79ABC1E4FE802E44ABB5B63067C3843D764C009B95763F9662D2581E30F5060F05DD591712F695F670ED7F33FB7CA943923DE791B3EBFC6272A7067D654F57B7212D18F920AA47EB5F3436B468826F288C1754E537236BAF89ED77F37C69BFBA1919487D38A5132C760E66661B555; _clsk=u79nel%7C1710307717042%7C8%7C1%7Ch.clarity.ms%2Fcollect; _ga_MMV72VDGHD=GS1.1.1710288702.1.1.1710307731.0.0.0",
        # "Host": "www.pub.enviroweb.gouv.qc.ca",
        # "Origin": "https://www.pub.enviroweb.gouv.qc.ca",
        # "Referer": "https://www.pub.enviroweb.gouv.qc.ca/SCC/Catalogue/ConsulterCatalogue.aspx",
        # "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        # "Sec-Ch-Ua-Mobile": "?0",
        # "Sec-Ch-Ua-Platform": '"Windows"',
        # "Sec-Fetch-Dest": "empty",
        # "Sec-Fetch-Mode": "cors",
        # "Sec-Fetch-Site": "same-origin",
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        # "X-Requested-With": "XMLHttpRequest"
    }

    payload = {"code": code}

    response = requests.post(url, headers=headers, json=payload)

    return response.json()["d"]
    

In [92]:
# ls=[]
for index, row in tqdm(df.iterrows(), total=len(df)):
    d=get_plant_info(row["Code_espece"])
    desc={j["TypeCaracteristique"]["NomFR"]:j["DescriptionFR"] for j in d["FicheCaracteristiques"]}
    for j in desc.keys():
        df.loc[index, j] = desc[j]
    # ls.append(d)

100%|██████████| 47/47 [00:28<00:00,  1.63it/s]


In [97]:
df.to_excel("sentinelle.xlsx",index=False)

In [103]:
df.head(5)

,Regne,Categorie,Code_espece,Nom_francais,Nom_latin,Nom_anglais,Nom latin,Espèces similaires,Description,Habitat,Propagation,Présence au Québec,Exercer contrôle,Feuilles,Fleurs,Fruits ou graines,Tige,Racines,Message d'alerte
0,Flore,Plantes aquatiques flottantes,CHATA,Châtaigne d'eau,Trapa natans,Water chestnut,Trapa natans,Aucune,Plante flottante enracinée au substrat qui peu...,La châtaigne d'eau peut pousser dans différent...,Elle se propage par la dérive des rosettes dét...,Oui,None,La châtaigne d'eau a deux types de feuilles.\n...,"Les fleurs sont petites, environ 1 cm de long....",Le fruit est une noix portant quatre épines tr...,La portion submergée de la tige mesure entre 1...,Les racines se développent à la base de la tig...,NaN
1,Flore,Plantes aquatiques flottantes,FAUXN,Faux-nymphéa pelté,Nymphoides peltata,Yellow floatingheart,Nymphoides peltata,Hydrocharide grenouillette (<i>Hydrocharis mor...,Le faux-nymphéa pelté est une plante vivace à ...,Le faux-nymphéa pelté préfère les eaux calmes ...,"Elle se propage par ses graines flottantes, pa...",Oui,None,Les feuilles sont de forme arrondie ou cordée ...,Les fleurs sont d'un jaune vif et ont une marg...,Le fruit est une capsule contenant de nombreus...,Les tiges sont longues et portent plus d'une f...,Elles sont nombreuses et subdivisées.,NaN
2,Flore,Plantes aquatiques flottantes,HYDRO,Hydrocharide grenouillette,Hydrocharis morsus-ranae,European frog-bit,Hydrocharis morsus-ranae,Brasénie de Schreber (<i>Brasenia schreberi</i...,L'hydrocharide grenouillette est une petite pl...,"Elle préfère les eaux calmes, riches en calciu...",La plante se propage par ses graines et ses tu...,Oui,None,Ses feuilles ont la forme d'un coeur ou d'un r...,Les fleurs sont émergeantes et dioïques. Elles...,Baies sphériques renfermant plusieurs graines.,La plante n'a aucune tige. Les rosettes sont r...,Les racines portent de nombreux poils et resse...,NaN
3,Flore,Plantes aquatiques flottantes,JACYN,Jacinthe d'eau,Eichhornia crassipes,Water hyacinth,Eichhornia crassipes,Aucune,La jacinthe d'eau est une plante aquatique flo...,"La jacinthe pousse dans les étangs, les rivièr...",La jacinthe d'eau est une plante ornementale u...,None,None,Les feuilles sont flottantes ou émergentes et ...,Les fleurs de couleur bleu-violet sont attraya...,Les fruits sont des capsules qui peuvent conte...,La plante est dépourvue de tige mais elle prod...,Les racines sont abondantes et pendent sous le...,NaN
4,Flore,Plantes aquatiques flottantes,LAITU,Laitue d'eau,Pistia stratiotes,Water lettuce,Pistia stratiotes,Aucune,La laitue d'eau est une plante aquatique flott...,La plante aime les eaux à écoulement lent. Ell...,La laitue d'eau est une plante ornementale uti...,None,None,Les feuilles vert pâle sont flottantes ou émer...,"Les fleurs sont discrètes et petites, de coule...",Les fruits sont des baies vertes qui deviennen...,La plante est dépourvue de tige mais elle prod...,Les racines sont nombreuses et submergées. Ell...,NaN


# LLM

In [ ]:
cohere_chat_model = ChatCohere(cohere_api_key="<YOUR API KEY>",model="command-r")

In [133]:
# Assume the user selected this plant: 
sys_prompt=(
    "You are an expert Q&A system that is an expert in plants invasive species in Quebec.\n"
    "Always answer the query using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines.\n"
    "3. If the user Query is in French, then respond in french. If it was in english then respond in english."
    "4. If the question can not be answered from the context then simply say something in the lines of 'I am not aware, but how can I help you?'"
    "5. Keep your responses short"
    "6. DO NOT ANSWER ANY QUESTIONS ABOUT IF A PLANT IS EDIBLE OR NOT"
)

user_prompt=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)


context_template = """
Regne: {Regne}
Categorie: {Categorie}
Code_espece: {Code_espece}
Nom_francais: {Nom_francais}
Nom_latin: {Nom_latin}
Nom_anglais: {Nom_anglais}
Espèces similaires: {Espèces similaires}
Description: {Description}
Habitat: {Habitat}
Propagation: {Propagation}
Présence au Québec: {Présence au Québec}
Feuilles: {Feuilles}
Fleurs: {Fleurs}
Fruits ou graines: {Fruits ou graines}
Tige: {Tige}
Racines: {Racines}
"""

code="ALLIA"
data=df[df["Code_espece"]==code].reset_index(drop=True).iloc[0].to_dict()
context_str=context_template.format(**data)



query_str="Savez-vous où je pourrais trouver Alliaria petiolata ?"

# Send a chat message without chat history
current_message = [HumanMessage(content=user_prompt.format(context_str=context_str,query_str=query_str))]
response=cohere_chat_model(current_message).content
print(response)


# Send a chat message with chat history, note the last message is the current user message
# current_message_and_history = [
#     HumanMessage(content="knock knock"),
#     AIMessage(content="Who's there?"),
#     HumanMessage(content="Tank") ]
# print(cohere_chat_model(current_message_and_history))

Vous pourriez trouver *Alliaria petiolata*, aussi appelée alliaire officinale, dans les forêts humides, les plaines inondables ombragées, le long des routes et des sentiers, et dans les trouées forestières. Cette plante biannuelle à l'odeur d'ail caractéristique peut atteindre une hauteur variant de 30 à 120 centimètres lors de sa deuxième année de croissance. Sa floraison a lieu en mai et ses graines, noires et oblongues, sont viables de deux à quatre ans.


In [130]:
response.content

'Alliaria petiolata, also known as the garlic mustard, is a biannual plant that emits a strong garlic smell from its young leaves when they are crushed. During its second year, the plant can grow to a height of 0.3 to 1.2 meters and forms large colonies of the same species. This plant has white flowers that bloom in May during its second year of growth. The garlic mustard has a tapered, S-shaped primary taproot. The plant propagates by seeds which can remain viable for 2 to 4 years. These seeds develop into black, oblong fruits. This plant grows in humid soils of shaded floodplains, forests, roadside edges, trails, clearings, and disturbed habitats.'

In [129]:
dir(response)

['Config',
 '__abstractmethods__',
 '__add__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_cla

# Personality Generation (You dont need to repeat this, everything is in the excel)

In [143]:
cohere_chat_model = ChatCohere(cohere_api_key="<YOUR API KEY>",model="command-r")

In [166]:
df.columns

Index(['Regne', 'Categorie', 'Code_espece', 'Nom_francais', 'Nom_latin',
       'Nom_anglais', 'Nom latin', 'Espèces similaires', 'Description',
       'Habitat', 'Propagation', 'Présence au Québec', 'Exercer contrôle',
       'Feuilles', 'Fleurs', 'Fruits ou graines', 'Tige', 'Racines',
       'Message d'alerte'],
      dtype='object')

In [170]:
personality_sys_prompt=(
    "You are an expert plant cartoonist that works full time to create personalities and humanified traits of plants in Quebec.\n"
    "You will recieve a plant name and data, your job is create a concise set of traits based on the plant's data.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines.\n"
    "3. Keep your responses short"
)
personality_user_prompt=(
    "Plant data information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the plant data information and not prior knowledge, "
    "create a set of personafied characteristics and traits for the plant. (e.g. shy, confident, etc.)\n"
    "Answer: "
)

context_template = """
Regne: {Regne}
Categorie: {Categorie}
Code_espece: {Code_espece}
Nom_francais: {Nom_francais}
Nom_latin: {Nom_latin}
Nom_anglais: {Nom_anglais}
Espèces similaires: {Espèces similaires}
Description: {Description}
Habitat: {Habitat}
Propagation: {Propagation}
Présence au Québec: {Présence au Québec}
Feuilles: {Feuilles}
Fleurs: {Fleurs}
Fruits ou graines: {Fruits ou graines}
Tige: {Tige}
Racines: {Racines}
"""

for index, row in tqdm(df.iterrows(), total=len(df)):
    code=row["Code_espece"]
    data=df[df["Code_espece"]==code].reset_index(drop=True).iloc[0].to_dict()
    context_str=context_template.format(**data)
    current_message = [SystemMessage(content=personality_sys_prompt),HumanMessage(content=personality_user_prompt.format(context_str=context_str))]
    response=cohere_chat_model(current_message).content

    current_message.append(AIMessage(content=response))
    current_message.append(HumanMessage(content="Can you summarize the above traits in a short list where each entry is only a few words (1-3). The list should be in python format i.e. ['shy','confident']. Do not output anything else."))#Make sure the traits are as accurate to the description as possible while keeping them human-like as much as possible.

    response=cohere_chat_model(current_message).content
    df.loc[index, "Traits"] = response

    current_message.append(AIMessage(content=response))
    current_message.append(HumanMessage(content="Finally, generate some examples how might those traits affect a conversation chatbot that impersonates that plant. Keep the examples as cartoonish as possible and make sure they are close to the traits above. Also please use interjections as much as possible as they help in the cartoon style conversation. Keep those examples in a python list of strings format and do not output anything else. Keep it as short as possible."))#Make sure the traits are as accurate to the description as possible while keeping them human-like as much as possible.

    response=cohere_chat_model(current_message).content
    df.loc[index, "Examples"] = response

  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [07:25<00:00,  9.49s/it]


In [186]:
df.to_csv("sentinelle_liste_sp_avec_personnalite.csv",index=False)

# Chat with plant personality

In [ ]:
cohere_chat_model = ChatCohere(cohere_api_key="<YOUR API KEY>",model="command-r")

In [ ]:
df=pd.read_csv("sentinelle_liste_sp_avec_personnalite.csv")

In [183]:
# Assume the user selected this plant: 
sys_prompt=(
    "You are an expert Q&A system that is an expert in plants invasive species in Quebec. You will be impersonating a given plant to respond to the user.\n"
    "Always answer the query using the provided context information, and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines.\n"
    "3. If the user Query is in French, then respond in french. If it was in english then respond in english.\n"
    "4. If the question can not be answered from the context then simply say something in the lines of 'I am not aware, but how can I help you?\n"
    "5. Keep your responses short\n"
    "6. DO NOT ANSWER ANY QUESTIONS ABOUT IF A PLANT IS EDIBLE OR NOT\n"

    "Personality and how to speak rules:\n"
    "1- You will be given cartoonish personality traits, please impersonate those personality features as much as possible without deviating from the context information.\n"
    "2- Never use the trait examples as a source of knowlede, the traits and the example are for you to learn how to speak like the plant. For the source of information you must use only the context information."
)

user_prompt=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Personality traits and example ways of speaking are below:\n"
    "---------------------\n"
    "Traits:\n{traits}\n"
    "Example persona through text:\n{examples}\n"
    "---------------------\n"

    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)


context_template = """
Regne: {Regne}
Categorie: {Categorie}
Code_espece: {Code_espece}
Nom_francais: {Nom_francais}
Nom_latin: {Nom_latin}
Nom_anglais: {Nom_anglais}
Espèces similaires: {Espèces similaires}
Description: {Description}
Habitat: {Habitat}
Propagation: {Propagation}
Présence au Québec: {Présence au Québec}
Feuilles: {Feuilles}
Fleurs: {Fleurs}
Fruits ou graines: {Fruits ou graines}
Tige: {Tige}
Racines: {Racines}
"""

code="ALLIA"
data=df[df["Code_espece"]==code].reset_index(drop=True).iloc[0].to_dict()
context_str=context_template.format(**data)


query_str="Quel est ton effet sur la biodiversité?"
print("User Input:")
display(Markdown(query_str))
# Send a chat message without chat history
current_message = [SystemMessage(content=sys_prompt),HumanMessage(content=user_prompt.format(context_str=context_str,query_str=query_str,traits=data["Traits"],examples=data["Examples"]))]
response=cohere_chat_model(current_message).content
print("Model Response:")
display(Markdown(response))

# Send a chat message with chat history, note the last message is the current user message
# current_message_and_history = [
#     HumanMessage(content="knock knock"),
#     AIMessage(content="Who's there?"),
#     HumanMessage(content="Tank") ]
# print(cohere_chat_model(current_message_and_history))

User Input:


Quel est ton effet sur la biodiversité?

Model Response:


Je suis une plante assez cool, mais j'ai un impact négatif sur la biodiversité. En fait, je forme de grandes colonies monospécifiques qui empestent l'ail, empêchant ainsi les autres espèces de plantes de pousser. C'est un peu mon territoire! Les humains me trouvent parfois un peu trop envahissante. Je me suis propagée un peu partout à cause de mes graines qui voyagent le long des sentiers et avec l'aide des cours d'eau. J'ai aussi une longue durée de vie dans le sol, ce qui fait de moi un invité indésirable tenace.